In [2]:
!pip3 install mediapipe opencv-python pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np

mp_drawing = mp.solutions.drawing_utils # drawing helpers
mp_holistic = mp.solutions.holistic # landmark detection models

cap = cv2.VideoCapture(0)

# initiate holistic model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # make detections based on recoloured feed
        results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # draw face landmarks
        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

        # draw right hand landmarks
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw left hand landmarks
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw pose landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # make detections
        cv2.imshow('holistic model detections', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1716430041.349131 7368449 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1716430041.465451 7368581 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430041.476435 7368586 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430041.478439 7368584 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430041.478448 7368582 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430041.478935 7368583 inference_feedback_manager.cc:114] Feedback manager requires a model

KeyboardInterrupt: 

In [5]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
landmarks = ['pose']
for i in range(1, num_coords+1):
    landmarks += ['x{}'.format(i), 'y{}'.format(i), 'z{}'.format(i), 'v{}'.format(i)]

csv_writer = csv.writer(open('coords.csv', mode='w', newline=''), delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
csv_writer.writerow(landmarks)

9594

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv

mp_drawing = mp.solutions.drawing_utils # drawing helpers
mp_holistic = mp.solutions.holistic # landmark detection models

pose_name = "emotionless"
cap = cv2.VideoCapture(0)

# initiate holistic model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # make detections based on recoloured feed
        results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # draw face landmarks
        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

        # draw right hand landmarks
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw left hand landmarks
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw pose landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # export coordinates
        try:
            # extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            csv_writer = csv.writer(open('coords.csv', mode='a', newline=''), delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow([pose_name] + pose_row + face_row)

        except:
            pass

        # make detections
        cv2.imshow('holistic model detections', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1716430220.109989 7370631 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1716430220.253579 7371420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430220.266231 7371420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430220.269342 7371419 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430220.269336 7371420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430220.269891 7371414 inference_feedback_manager.cc:114] Feedback manager requires a model

KeyboardInterrupt: 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('coords.csv')

feat = df.drop('pose', axis=1) # all but first col
targ = df['pose'] # first col

feat_train, feat_test, targ_train, targ_test = train_test_split(feat, targ, test_size=0.3, random_state=1234)

In [3]:
df.head()

,pose,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,shock,0.625259,0.488210,-0.506665,0.999990,0.655957,0.419267,-0.471772,0.999976,0.670879,...,-0.002376,0.0,0.680587,0.427756,0.005928,0.0,0.686105,0.420578,0.005900,0.0
1,shock,0.617741,0.513095,-0.536126,0.999990,0.650620,0.434174,-0.509820,0.999973,0.667857,...,-0.004143,0.0,0.671997,0.440529,0.002907,0.0,0.677512,0.433748,0.002687,0.0
2,shock,0.615368,0.530661,-0.532332,0.999990,0.648318,0.446826,-0.503713,0.999973,0.666445,...,-0.005700,0.0,0.670444,0.442035,0.000257,0.0,0.675703,0.435932,-0.000181,0.0
3,shock,0.612365,0.538415,-0.543973,0.999989,0.644232,0.453737,-0.518162,0.999971,0.663392,...,-0.005395,0.0,0.669133,0.448209,0.000415,0.0,0.674554,0.440665,0.000153,0.0
4,shock,0.612730,0.540112,-0.540258,0.999989,0.644534,0.455702,-0.512951,0.999970,0.663492,...,-0.004440,0.0,0.672703,0.454840,0.002630,0.0,0.678015,0.446508,0.002567,0.0


In [4]:
df.tail()

,pose,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
523,emotionless,0.659273,0.605414,-0.993543,0.999724,0.666334,0.536881,-0.950971,0.999188,0.676328,...,0.002029,0.0,0.692779,0.519000,0.036285,0.0,0.695221,0.512679,0.038389,0.0
524,emotionless,0.670889,0.607428,-1.085822,0.999735,0.677520,0.537703,-1.043021,0.999231,0.687089,...,0.002399,0.0,0.698708,0.517990,0.039551,0.0,0.700654,0.511061,0.041991,0.0
525,emotionless,0.682681,0.607666,-0.982500,0.999712,0.687199,0.539192,-0.941686,0.999198,0.694559,...,0.002003,0.0,0.706535,0.514982,0.041727,0.0,0.708358,0.508194,0.044360,0.0
526,emotionless,0.690894,0.607655,-0.942443,0.999657,0.692222,0.539315,-0.904090,0.999097,0.698523,...,0.002352,0.0,0.711501,0.518049,0.042844,0.0,0.713171,0.511160,0.045578,0.0
527,emotionless,0.697242,0.607701,-0.866478,0.999627,0.698804,0.539647,-0.831656,0.999050,0.704857,...,0.002903,0.0,0.714658,0.515532,0.044254,0.0,0.716132,0.508685,0.047074,0.0


In [5]:
from sklearn.pipeline import make_pipeline # to make ML pipeline
from sklearn.preprocessing import StandardScaler # to standardize data
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

fit_models= {}
for algo, pipeline in pipelines.items():
    fit_models[algo] = pipeline.fit(feat_train, targ_train)

In [6]:
fit_models['rf'].predict(feat_test)

array(['shock', 'emotionless', 'shock', 'shock', 'emotionless', 'shock',
       'shock', 'shock', 'shock', 'shock', 'shock', 'shock', 'shock',
       'shock', 'shock', 'shock', 'shock', 'shock', 'emotionless',
       'shock', 'shock', 'shock', 'emotionless', 'shock', 'emotionless',
       'shock', 'shock', 'emotionless', 'shock', 'shock', 'shock',
       'shock', 'emotionless', 'shock', 'emotionless', 'shock', 'shock',
       'emotionless', 'shock', 'shock', 'shock', 'shock', 'shock',
       'shock', 'shock', 'shock', 'shock', 'shock', 'shock', 'shock',
       'shock', 'shock', 'shock', 'shock', 'emotionless', 'shock',
       'emotionless', 'shock', 'emotionless', 'shock', 'shock', 'shock',
       'shock', 'shock', 'shock', 'shock', 'shock', 'shock', 'shock',
       'emotionless', 'shock', 'shock', 'shock', 'shock', 'shock',
       'shock', 'emotionless', 'shock', 'shock', 'shock', 'shock',
       'emotionless', 'shock', 'shock', 'emotionless', 'emotionless',
       'shock', 'emotionle

In [7]:
from sklearn.metrics import accuracy_score # accuracy metrics
import pickle

for algo, model in fit_models.items():
    yhat = model.predict(feat_test)
    print(algo, accuracy_score(targ_test, yhat)) # print accuracy

pickle.dump(fit_models['rf'], open('body_language.pkl', 'wb'))
model = pickle.load(open('body_language.pkl', 'rb'))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd
import pickle

mp_drawing = mp.solutions.drawing_utils # drawing helpers
mp_holistic = mp.solutions.holistic # landmark detection models

model = pickle.load(open('body_language.pkl', 'rb'))

cap = cv2.VideoCapture(0)

# initiate holistic model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # make detections based on recoloured feed
        results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # draw face landmarks
        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

        # draw right hand landmarks
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw left hand landmarks
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # draw pose landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # export coordinates
        try:
            # extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row

            feat = pd.DataFrame([row])
            bl_class = model.predict(feat)[0]
            bl_prob = model.predict_proba(feat)[0]

            # get status box
            cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (234,244,232), 1, cv2.LINE_AA)
            cv2.putText(frame, bl_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (190,216,184), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(frame, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (234,244,232), 1, cv2.LINE_AA)
            cv2.putText(frame, str(round(bl_prob[np.argmax(bl_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (190,216,184), 2, cv2.LINE_AA)



        except:
            pass

        # make detections
        cv2.imshow('holistic model detections', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1716430559.630812 7375194 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
W0000 00:00:1716430559.701982 7375434 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430559.711116 7375431 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430559.712237 7375432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430559.712245 7375433 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716430559.712662 7375435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support fo

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 